<a href="https://colab.research.google.com/github/mamoan/NOVA_DL_home_exercise/blob/main/4_model_training_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4 - Training a YOLOv8 object detection model on the small dataset
#    🖼️ +📝 --> 💻🧠

**OBJECTIVE:** The objective is to train YOLOv8 **object detection** models on a small dataset annotated by me ☝️. I tested different hyperparameters.

In [ ]:
annotator_ID=8 # change this to your folder ID

path_to_tiles="/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/"+str(annotator_ID)

### 4.1 Switch on the GPU
GPUs speed up training time when using GPUs, and google colab offers GPU for free, although it does not offer unlimited GPU.

### 4.2 Setup YOLOv8, comet, and import other libraries

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check software and hardware. Here I also load [Comet](https://www.comet.com/site/lp/yolov5-with-comet/?utm_source=yolov8&utm_medium=partner&utm_content=github) which will be used to track the model performances.

In [ ]:
%pip install comet_ml --quiet
import comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.7/534.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.31.0 which is incompatible.


Start Comet project

In [ ]:
comet_ml.init(project_name='seedling_detection_YOLOv8_full_data')

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
import yaml

Ultralytics YOLOv8.0.118 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.1/78.2 GB disk)


In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 4.3 Create configuration file (.yaml) to pass for model training

In [ ]:
config_data = {
    'path': path_to_tiles,
    'train': path_to_tiles+'/train/images',
    'val': path_to_tiles+'/val/images',
    'test':"/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/test/images",
    'names': {
        0: 'tree'
    }
}

In [ ]:
with open(path_to_tiles+'/train_config.yaml', 'w') as file:
    yaml.dump(config_data, file)

### 4.4 Train

Model training through the command line interface (CLI). This tutorial: https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results has aided me in the model training process.

I started with adjusting just two options, model size and image size:

- *model* The larger the model the better the performance but also the computational resources required to train and predict. For seedling detection, I think it is logical that the model should either be deployed locally, i.e. the person operating the drone can run the model themselves on their Ipad, or that the operator uploads the images to a server. In this latter case, the model can be larger. I will try model sizes small (yolov8s.pt) and large (yolov8l.pt). I am not fitting an extra large model, to limit the risk that I run out of computing power.
- *imgsz*: Image size. I tried image sizes 640 and 1240. I should remember to also use the same resolution while testing.


### Small model, image size: 640

In [ ]:
path_to_tiles

'/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8'

In [ ]:
!yolo train model=yolov8s.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=640 project=$path_to_tiles name="seedlings_YOLOs_img640_small" seed= 130623

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8/train_config.yaml, epochs=300, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8, name=seedlings_YOLOs_img640_small, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=130623, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=N

### Small model, image size: 1240

In [ ]:
!yolo train model=yolov8s.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=1240 project=$path_to_tiles name="seedlings_YOLOs_img1240_small" seed= 130623

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8/train_config.yaml, epochs=300, patience=50, batch=16, imgsz=1240, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8, name=seedlings_YOLOs_img1240_small, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=130623, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width

### Large model, image size: 640

In [ ]:
!yolo train model=yolov8l.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=640 project=$path_to_tiles name="seedling_YOLOl_img640_small" seed= 130623

100% 83.7M/83.7M [00:00<00:00, 199MB/s]
Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8/train_config.yaml, epochs=300, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8, name=seedling_YOLOl_img640_small, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=130623, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

### Large model, image size: 1240

In [ ]:
# !yolo train model=yolov8l.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=1240 project=$path_to_tiles name="seedlings_YOLOl_img1240_small" seed= 130623

I was supposed to run the model above, but I got this error in Comet:

"Unexpected error detected in the user script: OutOfMemoryError('CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.75 GiB total capacity; 14.50 GiB already allocated; 2.81 MiB free; 14.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation. See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')"

Therefore, I instead tried to run the model with a smaller batch size: batch size 10.

In [ ]:
import torch
torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
!yolo train model=yolov8l.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=1240 project=$path_to_tiles name="seedlings_YOLOl_img1240_small" batch= 5 seed= 130623

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8/train_config.yaml, epochs=300, patience=50, batch=5, imgsz=1240, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8, name=seedlings_YOLOl_img1240_small, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=130623, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=

The result of this model was not good. Probably because of the batch size being so small



### Further hyperparameter tuning

After these initial models, I found out that the smaller models seemed to perform the best. Although the model with image size 1240 seemed to perform slightly better than the model with image size 640, I wanted to test the image size 1056 as well to see if this could give a similar improvement to using 1240. Therefore, I also tried a yolo nano model with image size 1056 to see if an even smaller model might yield a better result.


In [ ]:
!yolo train model=yolov8n.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=1056 project=$path_to_tiles name="seedling_YOLOl_img1056_nano" seed= 130623

The nano model did worse. I then tested the small model with an image size of 1056 and I also wanted to try to changing the learning rate since it [seems to be important](https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/). I also wanted to try using [autobatch](https://docs.ultralytics.com/reference/yolo/utils/autobatch/#ultralytics.yolo.utils.autobatch.check_train_batch_size) which can be done by setting batch = -1.

In [ ]:
!yolo train model=yolov8s.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=1056 project=$path_to_tiles name="seedling_YOLOl_img1056_nano" batch= -1 seed= 130623

100% 21.5M/21.5M [00:01<00:00, 21.7MB/s]
Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8/train_config.yaml, epochs=300, patience=50, batch=-1, imgsz=1056, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/train/8, name=seedling_YOLOl_img1056_nano, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=130623, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, s

This is the best model yet.

### I then trained models for the full dataset where I somewhat based my choices on lessons learned from this smaller dataset.